In [2]:
!pip install opencv-python
!pip install pygame

     ---------------------------------------- 10.7/10.7 MB 5.1 MB/s eta 0:00:00


In [1]:
import cv2
import numpy as np
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.7.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\nickn\AppData\Local\anaconda3\envs\niyath37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\nickn\AppData\Local\anaconda3\envs\niyath37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\nickn\AppData\Local\anaconda3\envs\niyath37\lib\site-packages\numpy\.libs\libopenblas.wcdjnk7yvmpzq2me2zzhjjrj3jikndb7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
pygame.mixer.init()

In [3]:
drum_sounds = {
    'snare': pygame.mixer.Sound('snare.wav'),
    'kick': pygame.mixer.Sound('kick.wav'),
    'hihat': pygame.mixer.Sound('hihat.wav')
}

In [4]:
def is_inside_rect(point, rect):
    x, y = point
    rx, ry, rw, rh = rect
    return rx < x < rx + rw and ry < y < ry + rh


In [5]:
def play_drum_sound(gesture, velocity):
    if gesture in drum_sounds:
        # Adjust volume based on velocity
        volume = max(min(velocity / 10, 1.0), 0.3)  # Scale velocity to a range between 0.3 and 1.0
        drum_sounds[gesture].set_volume(volume)
        drum_sounds[gesture].play()

In [6]:
roi_coords = {
    'left_hand': (50, 50, 200, 200),   # (x, y, width, height)
    'right_hand': (395, 50, 200, 200)
}

In [7]:
cap = cv2.VideoCapture(0)

# Previous centroid positions for velocity calculation for left and right hands
prev_cx_left = None
prev_cy_left = None
prev_cx_right = None
prev_cy_right = None

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)

    # Draw ROIs
    for roi_name, coords in roi_coords.items():
        x, y, w, h = coords
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (15, 15), 0)

    # Threshold the image to get binary image
    _, thresh = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate through contours
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 1000:  # Adjust this threshold according to your environment
            # Get center of contour
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])

                # Calculate velocity based on previous centroid position for each hand
                if 'left_hand' in roi_coords and is_inside_rect((cx, cy), roi_coords['left_hand']):
                    if prev_cx_left is not None and prev_cy_left is not None:
                        velocity_left = np.sqrt((cx - prev_cx_left) ** 2 + (cy - prev_cy_left) ** 2)
                    else:
                        velocity_left = 0
                    prev_cx_left = cx
                    prev_cy_left = cy
                    play_drum_sound('hihat', velocity_left)  # Play sound for left hand
                elif 'right_hand' in roi_coords and is_inside_rect((cx, cy), roi_coords['right_hand']):
                    if prev_cx_right is not None and prev_cy_right is not None:
                        velocity_right = np.sqrt((cx - prev_cx_right) ** 2 + (cy - prev_cy_right) ** 2)
                    else:
                        velocity_right = 0
                    prev_cx_right = cx
                    prev_cy_right = cy
                    play_drum_sound('kick', velocity_right)  # Play sound for right hand

                # Draw circle
                cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)

    # mask (ROI)
    mask = np.zeros_like(frame)
    for roi_name, coords in roi_coords.items():
        x, y, w, h = coords
        cv2.rectangle(mask, (x, y), (x + w, y + h), (255, 255, 255), -1)
    masked_frame = cv2.bitwise_and(frame, mask)

    cv2.imshow('Air Drums', masked_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [87]:
# roi_right_hand = (50, 50, 200, 200)   # (x, y, width, height)

# # Function to check if a point (hand) is inside a rectangle (ROI)
# def is_inside_rect(point, rect):
#     x, y = point
#     rx, ry, rw, rh = rect
#     return rx < x < rx + rw and ry < y < ry + rh

# # Function to play drum sound based on gesture
# def play_drum_sound(gesture):
#     if gesture in drum_sounds:
#         drum_sounds[gesture].play()

# # OpenCV setup
# cap = cv2.VideoCapture(0)

# # Previous state of the right hand (open or closed)
# prev_right_hand_state = None

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         breakq

#     # Flip the frame horizontally
#     frame = cv2.flip(frame, 1)

#     # Draw ROI for the right hand on the frame
#     x, y, w, h = roi_right_hand
#     cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

#     # Convert frame to grayscale
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Apply Gaussian blur
#     blurred = cv2.GaussianBlur(gray, (15, 15), 0)

#     # Threshold the image to get binary image
#     _, thresh = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)

#     # Find contours
#     contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#     # Check if the right hand is open or closed
#     right_hand_state = 'open' if len(contours) > 0 else 'closed'

#     # If the state of the right hand has changed
#     if right_hand_state != prev_right_hand_state:
#         # Determine the gesture based on the state of the right hand
#         gesture = 'snare' if right_hand_state == 'open' else 'kick'

#         # Play drum sound corresponding to the gesture
#         play_drum_sound(gesture)

#         # Update the previous state of the right hand
#         prev_right_hand_state = right_hand_state

#     cv2.imshow('Air Drums', frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()